# Twitterbot

Here I am going to create a Twitter Bot step by step.

First I should create a Twitter application on https://dev.twitter.com/

Install tweepy: #pip install tweepy

In [2]:
import tweepy
import time
import sys
import pickle
import datetime

Twitter application information:

In [69]:
filename="auth_data"
fileobject=open(path+filename,"rb")
auth_data=pickle.load(fileobject)
fileobject.close()

In [71]:
#account=auth_data1['account']
CONSUMER_KEY=auth_data1['CONSUMER_KEY']
CONSUMER_SECRET=auth_data1['CONSUMER_SECRET']
ACCESS_KEY=auth_data1['ACCESS_KEY']
ACCESS_SECRET=auth_data1['ACCESS_SECRET']

In [72]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tweepy.API(auth)

In [73]:
#Error handling
if (not api):
    print ("Problem connecting to API")

# Search Parameters

q: the search query string
lang: Restricts tweets to the given language, given by an ISO 639-1 code. (fa for persian)
locale: Specify the language of the query you are sending. This is intended for language-specific clients and the default should work in the majority of cases.

In [74]:
filename="users_farsi"
path="./data/"

In [34]:
users=set()
requestnum=0

In [75]:
fileobject=open(path+filename,"rb")
users=pickle.load(fileobject)
fileobject.close()

In [76]:
# In this example, the handler is time.sleep(15 * 60),
# but you can of course handle it in any way you want.

def limit_handled(c):
    while True:
        try:
            yield c.next()
        except tweepy.RateLimitError:
            print ("rate limit error")
            time.sleep(15 * 60)

In [ ]:
for t in limit_handled(tweepy.Cursor(api.search,q="*",lang="fa").items(1000)):
    users.add(t.user.id)

In [96]:
fileobject=open(path+filename,"wb")
pickle.dump(users,fileobject)
fileobject.close()

In [95]:
len(users)

3077

In [87]:
now = datetime.datetime.now()
#time1=datetime.datetime(2017,6,14,0)
#time2=datetime.datetime(2017,6,15,0)
for uid in users:
    user=api.get_user(uid)
    #requestnum=requestnum+1
    #print(t.user.screen_name,uid)
    if (not user.protected and (now-user.created_at).days>30):
        #print("not protected and older than 30 days")
        for t in tweepy.Cursor(api.user_timeline,user_id=uid,include_entities=True).items(20):
            if (now-t.created_at).days>1:
                break
            elif (t.favorite_count>1000):
                print(t.user.screen_name)
                print(t.text)
                print ("https://twitter.com/statuses/"+str(t.id))

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [82]:
a=datetime.datetime(2017,6,14,0)
print(a)

2017-06-14 00:00:00


some intresting questions to think about
in average how long does it takes to a tweet get 1000 faves?
how can evaluate a chance of a tweet to became favestar? which parameters are important? time, number of followers of user ...

how to prevent RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

how to prevent to find a tweet several times

I think its good to search for tweets that created in last 24h and repeat this every 2 hour.